<a href="https://colab.research.google.com/github/ahardi2/data_analysis/blob/main/01.%20rfm_analysis/rfm_analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# load library

import pandas as pd

In [ ]:
# load the dataset
data = '/content/drive/MyDrive/Dataset/superstore_dataset2011-2015.csv'

dt = pd.read_csv(data, encoding='cp1252')
#dt.info()

In [ ]:
# list comprehension - https://www.w3schools.com/python/python_lists_comprehension.asp
dt.columns = [col.lower().replace(' ', '_') for col in dt.columns]
#dt.nunique()

In [ ]:
# select columns for rfm analysis
df = dt[['order_id', 'order_date', 'customer_id', 'sales']]

In [ ]:
#df.head()

In [ ]:
# convert order_date column dtype to dataframe
df['order_date'] = pd.to_datetime(df['order_date'],dayfirst=False, format='mixed')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 51290 entries, 0 to 51289
Data columns (total 4 columns):
 #   Column       Non-Null Count  Dtype         
---  ------       --------------  -----         
 0   order_id     51290 non-null  object        
 1   order_date   51290 non-null  datetime64[ns]
 2   customer_id  51290 non-null  object        
 3   sales        51290 non-null  float64       
dtypes: datetime64[ns](1), float64(1), object(2)
memory usage: 1.6+ MB


<ipython-input-65-8b9a6c07a818>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['order_date'] = pd.to_datetime(df['order_date'],dayfirst=False, format='mixed')


In [ ]:
#df['order_date']

In [ ]:
import datetime as dt
df['year'] = df['order_date'].dt.year

In [ ]:
df.head()

,order_id,order_date,customer_id,sales,year
0,AG-2011-2040,2011-01-01,TB-11280,408.300,2011
1,IN-2011-47883,2011-01-01,JH-15985,120.366,2011
2,HU-2011-1220,2011-01-01,AT-735,66.120,2011
3,IT-2011-3647632,2011-01-01,EM-14140,44.865,2011
4,IN-2011-47883,2011-01-01,JH-15985,113.670,2011


In [ ]:
wdf = df.loc[df['year']==2011]

In [ ]:
wdf.info()

<class 'pandas.core.frame.DataFrame'>
Index: 8998 entries, 0 to 51117
Data columns (total 5 columns):
 #   Column       Non-Null Count  Dtype         
---  ------       --------------  -----         
 0   order_id     8998 non-null   object        
 1   order_date   8998 non-null   datetime64[ns]
 2   customer_id  8998 non-null   object        
 3   sales        8998 non-null   float64       
 4   year         8998 non-null   int32         
dtypes: datetime64[ns](1), float64(1), int32(1), object(2)
memory usage: 386.6+ KB


### 1. Calculate RFM Values

Recency
- convert 'PurchaseDate' from object to datetime data-type

In [ ]:
# calculate rfm values
# set dictionary for data aggregation
agg_dict = {'order_id': 'count', 'order_date': 'max', 'sales': 'sum'}

# calculate rfm values
wdf_rfm = wdf.groupby('customer_id').agg(agg_dict).reset_index()
# rename calculation ouput columns as RFM
wdf_rfm.columns = ['customer_id', 'frequency', 'max_date', 'monetary']

In [ ]:
#wd_rfm.info()

In [ ]:
# max order
print(wdf_rfm['max_date'].max())
# set date of analysis (today) a day after max date
from datetime import datetime
today = datetime.strptime('2012-01-01', '%Y-%m-%d')
print(today)
# calculate recency values
wdf_rfm['recency'] = wdf_rfm['max_date'].apply(lambda x: (today-x).days)
# drop 'max_date' column
wdf_rfm.drop(['max_date'], axis=1, inplace=True)

2011-12-31 00:00:00
2012-01-01 00:00:00


In [ ]:
wdf_rfm.describe().round(1)

,frequency,monetary,recency
count,1309.0,1309.0,1309.0
mean,6.9,1726.1,91.5
std,5.2,2092.7,90.7
min,1.0,2.7,1.0
25%,3.0,277.0,19.0
50%,6.0,982.5,57.0
75%,10.0,2442.1,137.0
max,27.0,24824.4,364.0


### calculate rfm scores

In [ ]:
r_labels, f_labels, m_labels = range(4, 0, -1), range(1,5), range(1,5)

wdf_rfm['r_score'] = pd.qcut(wdf_rfm['recency'], q=4, labels=r_labels).astype(int)
wdf_rfm['f_score'] = pd.qcut(wdf_rfm['frequency'], q=4, labels=f_labels).astype(int)
wdf_rfm['m_score'] = pd.qcut(wdf_rfm['monetary'], q=4,labels=m_labels ).astype(int)

In [ ]:
wdf_rfm['rfm_sum'] = wdf_rfm['r_score'] + wdf_rfm['f_score'] + wdf_rfm['m_score']

In [ ]:
wdf_rfm['rfm_code'] = wdf_rfm['r_score'].astype(str) + wdf_rfm['f_score'].astype(str) + wdf_rfm['m_score'].astype(str)

In [ ]:
wdf_rfm.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1309 entries, 0 to 1308
Data columns (total 9 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   customer_id  1309 non-null   object 
 1   frequency    1309 non-null   int64  
 2   monetary     1309 non-null   float64
 3   recency      1309 non-null   int64  
 4   r_score      1309 non-null   int64  
 5   f_score      1309 non-null   int64  
 6   m_score      1309 non-null   int64  
 7   rfm_sum      1309 non-null   int64  
 8   rfm_code     1309 non-null   object 
dtypes: float64(1), int64(6), object(2)
memory usage: 92.2+ KB


In [ ]:
wdf_rfm.head()

,customer_id,frequency,monetary,recency,r_score,f_score,m_score,rfm_sum,rfm_code
0,AA-10315,12,1732.2965,38,3,4,3,10,343
1,AA-10375,4,752.8820,69,2,2,2,6,222
2,AA-10480,6,2082.0730,142,1,2,3,6,123
3,AA-10645,12,2115.1950,6,4,4,3,11,443
4,AA-315,1,4.5000,148,1,1,1,3,111


In [ ]:
wdf_rfm.describe()

,frequency,monetary,recency,r_score,f_score,m_score,rfm_sum
count,1309.000000,1309.000000,1309.000000,1309.000000,1309.000000,1309.000000,1309.000000
mean,6.873950,1726.089301,91.543927,2.505730,2.333079,2.498854,7.337662
std,5.180548,2092.674284,90.682010,1.115366,1.163532,1.118802,2.850070
min,1.000000,2.652000,1.000000,1.000000,1.000000,1.000000,3.000000
25%,3.000000,276.984000,19.000000,2.000000,1.000000,1.000000,5.000000
50%,6.000000,982.480000,57.000000,3.000000,2.000000,2.000000,7.000000
75%,10.000000,2442.053040,137.000000,4.000000,3.000000,3.000000,10.000000
max,27.000000,24824.439000,364.000000,4.000000,4.000000,4.000000,12.000000


### EDA RFM data

In [ ]:
# check for outlier
import matplotlib.pyplot as plt
import seaborn as sns
attributes = ['recency', 'frequency', 'monetary']

In [ ]:
wdf_rfm['f_score'].value_counts()

f_score
1    447
3    297
4    293
2    272
Name: count, dtype: int64

In [ ]:
wdf_rfm.groupby('m_score').agg({'monetary': ['count','min', 'max']})

monetary                       
           count       min          max
m_score                                
1            328     2.652    276.98400
2            327   277.060    982.48000
3            327   996.720   2442.05304
4            327  2452.862  24824.43900

In [ ]:
wdf_rfm.groupby(['rfm_sum', 'rfm_code']).count()

customer_id  frequency  monetary  recency  r_score  f_score  \
rfm_sum rfm_code                                                                
3       111               154        154       154      154      154      154   
4       112                48         48        48       48       48       48   
        121                15         15        15       15       15       15   
        211                68         68        68       68       68       68   
5       113                10         10        10       10       10       10   
        122                28         28        28       28       28       28   
        212                34         34        34       34       34       34   
        221                12         12        12       12       12       12   
        311                42         42        42       42       42       42   
6       114                 4          4         4        4        4        4   
        123                26         26        26       26       26       26   
        132                 5          5         5        5        5        5   
        213                 4          4         4        4        4        4   
        222                42         42        42       42       42       42   
        312                28         28        28       28       28       28   
        321                11         11        11       11       11       11   
        411                20         20        20       20       20       20   
7       124                 5          5         5        5        5        5   
        133                11         11        11       11       11       11   
        142                 1          1         1        1        1        1   
        214                 2          2         2        2        2        2   
        223                22         22        22       22       22       22   
        232                22         22        22       22       22       22   
        313                10         10        10       10       10       10   
        322                24         24        24       24       24       24   
        331                 2          2         2        2        2        2   
        412                19         19        19       19       19       19   
        421                 4          4         4        4        4        4   
8       134                 5          5         5        5        5        5   
        143                 3          3         3        3        3        3   
        224                 6          6         6        6        6        6   
        233                35         35        35       35       35       35   
        242                 3          3         3        3        3        3   
        323                20         20        20       20       20       20   
        332                26         26        26       26       26       26   
        413                 4          4         4        4        4        4   
        422                28         28        28       28       28       28   
9       144                 7          7         7        7        7        7   
        234                23         23        23       23       23       23   
        243                17         17        17       17       17       17   
        324                 6          6         6        6        6        6   
        333                38         38        38       38       38       38   
        342                 4          4         4        4        4        4   
        423                18         18        18       18       18       18   
        432                14         14        14       14       14       14   
10      244                41         41        41       41       41       41   
        334                27         27        27       27       27       27   
        343                25         25        2

In [ ]:
result = orders_data.groupby('customer_id').agg({'purch_amt': ['mean', 'min', 'max']})

NameError: name 'orders_data' is not defined

In [ ]:
#calculate range of frequency of each f_score values
wdf_rfm.loc[wdf_rfm['f_score']==1]['frequency']

In [ ]:
wd_rfm['recency'].plot(kind='box', title='recency')

In [ ]:
wd_eda = wd_rfm.loc[wd_rfm['recency']>250]

In [ ]:
wd_eda.head(10)

In [ ]:
wd_rfm['monetary'].plot(kind='box', title='monetary')

In [ ]:
wd_rfm['frequency'].plot(kind='box', title='frequency')

merge the RFM

In [ ]:
#df_rfm['recency'].plot.hist()
wd_rfm['monetary'].plot.hist()

In [ ]:
wd_rfm['recency'].plot(kind='box', title='recency')

calculate rfm scores

In [ ]:
#df_rfm['r_rank']= df_rfm['recency'].rank(ascending=False)



In [ ]:
#df_rfm.drop(columns='r_rank')

In [ ]:
df_rfm.describe()

In [ ]:
'''
import pandas as pd
import plotly.express as pt
import plotly.io as pio
import plotly.graph_objects as go
pio.templates.default = "plotly_white"

data = pd.read_csv("rfm_data.csv")
print(data.head())
'''

Calculating RFM values

Calculating RFM scores

1. Concatenate all scores into one, eg: 3-4-1 (R-F-M) and group customers according to those values. It gives 125 combinations which is difficult to depict and understand
2. Sum all scores into one, eg: 8 (3 + 4 + 1) and group customers according to this number. It creates 13 groups which is much easier to work with but it disregards relations between R & F & M
3. Segment customers based on set of rules applied to R-F-M scores. It creates 10 groups, such as "champions", "needing attention" or "at risk"

In [ ]:
### cannot group frequency as f_score as there are only 3 unique values in frequency
'''
r_labels, f_labels, m_labels = range(5,0,-1), range(1, 6), range(1, 6)

df_rfm['r_score'] = pd.qcut(df_rfm['recency'], q=5, labels=r_labels).astype(int)
df_rfm['f_score'] = pd.qcut(df_rfm['frequency'], q=5, labels=f_labels).astype(int)
df_rfm['m_score'] = pd.qcut(df_rfm['monetary'], q=5, labels=m_labels).astype(int)
df_rfm.head()
'''

In [ ]:
# normalize the RFM values
df_rfm['r_norm'] = ((1- df_rfm['recency']/df_rfm['recency'].max())*100).round(2)  # why di kali 100???
df_rfm['f_norm'] = ((df_rfm['frequency']/df_rfm['frequency'].max())*100).round(2)
df_rfm['m_norm'] = ((df_rfm['monetary']/df_rfm['monetary'].max())*100).round(2)

df_rfm.head()

In [ ]:
df_rfm['rfm_score'] = 0.4*df_rfm['r_norm'] + 0.2*df_rfm['f_norm'] + 0.4*df_rfm['m_norm']
df_rfm *= 0.05 # for setting the highest score = 5
df_rfm = df_rfm.round(2)
df_rfm.head()

In [ ]:
df_rfm['rfm_score'].max()

In [ ]:
df_rfm['rfm_score'].plot.hist()

In [ ]:
df_rfm['rfm_score'].describe()

The lower the recency the higher the recency score,

In [ ]:
rfm_df['RFM_Score'] = 0.15*rfm_df['R_rank_norm']+0.28 * \
    rfm_df['F_rank_norm']+0.57*rfm_df['M_rank_norm']
rfm_df['RFM_Score'] *= 0.05
rfm_df = rfm_df.round(2)
rfm_df[['CustomerName', 'RFM_Score']].head(7)

In [ ]:
'''
r_labels, f_labels, m_labels = range(5, 0, -1), range(1,6), range(1,6)

df_rfm['r_score'] = pd.qcut(df_rfm['recency'], q=5, labels=r_labels).astype(int)
df_rfm['f_score'] = pd.qcut(df_rfm['frequency'], q=5, labels=f_labels).astype(int)
df_rfm['m_score'] = pd.qcut(df_rfm['monetary'], q=5, labels=m_labels).astype(int)

df_rfm['rfm_sum'] = df_rfm['r_score'] + df_rfm['f_score'] + df_rfm['m_score']
'''

In [ ]:
rfm_df['R_rank'] = rfm_df['Recency'].rank(ascending=False)
rfm_df['F_rank'] = rfm_df['Frequency'].rank(ascending=True)
rfm_df['M_rank'] = rfm_df['Monetary'].rank(ascending=True)

# normalizing the rank of the customers
rfm_df['R_rank_norm'] = (rfm_df['R_rank']/rfm_df['R_rank'].max())*100
rfm_df['F_rank_norm'] = (rfm_df['F_rank']/rfm_df['F_rank'].max())*100
rfm_df['M_rank_norm'] = (rfm_df['F_rank']/rfm_df['M_rank'].max())*100

rfm_df.drop(columns=['R_rank', 'F_rank', 'M_rank'], inplace=True)

rfm_df.head()


In [ ]:
'''
# Define scoring criteria for each RFM value
recency_scores = [5, 4, 3, 2, 1]  # Higher score for lower recency (more recent)
frequency_scores = [1, 2, 3, 4, 5]  # Higher score for higher frequency
monetary_scores = [1, 2, 3, 4, 5]  # Higher score for higher monetary value

# Calculate RFM scores
data['RecencyScore'] = pd.cut(data['Recency'], bins=5, labels=recency_scores)
data['FrequencyScore'] = pd.cut(data['Frequency'], bins=5, labels=frequency_scores)
data['MonetaryScore'] = pd.cut(data['MonetaryValue'], bins=5, labels=monetary_scores)
'''

In [ ]:
'''
# Convert RFM scores to numeric type
data['RecencyScore'] = data['RecencyScore'].astype(int)
data['FrequencyScore'] = data['FrequencyScore'].astype(int)
data['MonetaryScore'] = data['MonetaryScore'].astype(int)
'''

RFM values segmentation

In [ ]:
'''
# Calculate RFM score by combining the individual scores
data['RFM_Score'] = data['RecencyScore'] + data['FrequencyScore'] + data['MonetaryScore']

# Create RFM segments based on the RFM score
segment_labels = ['Low-Value', 'Mid-Value', 'High-Value']
data['Value Segment'] = pd.qcut(data['RFM_Score'], q=3, labels=segment_labels)

'''

In [ ]:
'''
print(data.head())
'''

In [ ]:
'''
# RFM Segment Distribution
segment_counts = data['Value Segment'].value_counts().reset_index()
segment_counts.columns = ['Value Segment', 'Count']

pastel_colors = px.colors.qualitative.Pastel

# Create the bar chart
fig_segment_dist = px.bar(segment_counts, x='Value Segment', y='Count',
                          color='Value Segment', color_discrete_sequence=pastel_colors,
                          title='RFM Value Segment Distribution')

# Update the layout
fig_segment_dist.update_layout(xaxis_title='RFM Value Segment',
                              yaxis_title='Count',
                              showlegend=False)

# Show the figure
fig_segment_dist.show()

'''

RFM custumer segment

In [ ]:
'''
# Create a new column for RFM Customer Segments
data['RFM Customer Segments'] = ''

# Assign RFM segments based on the RFM score
data.loc[data['RFM_Score'] >= 9, 'RFM Customer Segments'] = 'Champions'
data.loc[(data['RFM_Score'] >= 6) & (data['RFM_Score'] < 9), 'RFM Customer Segments'] = 'Potential Loyalists'
data.loc[(data['RFM_Score'] >= 5) & (data['RFM_Score'] < 6), 'RFM Customer Segments'] = 'At Risk Customers'
data.loc[(data['RFM_Score'] >= 4) & (data['RFM_Score'] < 5), 'RFM Customer Segments'] = "Can't Lose"
data.loc[(data['RFM_Score'] >= 3) & (data['RFM_Score'] < 4), 'RFM Customer Segments'] = "Lost"

# Print the updated data with RFM segments
print(data[['CustomerID', 'RFM Customer Segments']])
'''

RFM analysis

In [ ]:
'''
segment_product_counts = data.groupby(['Value Segment', 'RFM Customer Segments']).size().reset_index(name='Count')

segment_product_counts = segment_product_counts.sort_values('Count', ascending=False)

fig_treemap_segment_product = px.treemap(segment_product_counts,
                                         path=['Value Segment', 'RFM Customer Segments'],
                                         values='Count',
                                         color='Value Segment', color_discrete_sequence=px.colors.qualitative.Pastel,
                                         title='RFM Customer Segments by Value')
fig_treemap_segment_product.show()
'''

In [ ]:
'''
# Filter the data to include only the customers in the Champions segment
champions_segment = data[data['RFM Customer Segments'] == 'Champions']

fig = go.Figure()
fig.add_trace(go.Box(y=champions_segment['RecencyScore'], name='Recency'))
fig.add_trace(go.Box(y=champions_segment['FrequencyScore'], name='Frequency'))
fig.add_trace(go.Box(y=champions_segment['MonetaryScore'], name='Monetary'))

fig.update_layout(title='Distribution of RFM Values within Champions Segment',
                  yaxis_title='RFM Value',
                  showlegend=True)

fig.show()
'''

In [ ]:
'''
correlation_matrix = champions_segment[['RecencyScore', 'FrequencyScore', 'MonetaryScore']].corr()

# Visualize the correlation matrix using a heatmap
fig_heatmap = go.Figure(data=go.Heatmap(
                   z=correlation_matrix.values,
                   x=correlation_matrix.columns,
                   y=correlation_matrix.columns,
                   colorscale='RdBu',
                   colorbar=dict(title='Correlation')))

fig_heatmap.update_layout(title='Correlation Matrix of RFM Values within Champions Segment')

fig_heatmap.show()
'''

In [ ]:
'''
import plotly.colors

pastel_colors = plotly.colors.qualitative.Pastel

segment_counts = data['RFM Customer Segments'].value_counts()

# Create a bar chart to compare segment counts
fig = go.Figure(data=[go.Bar(x=segment_counts.index, y=segment_counts.values,
                            marker=dict(color=pastel_colors))])

# Set the color of the Champions segment as a different color
champions_color = 'rgb(158, 202, 225)'
fig.update_traces(marker_color=[champions_color if segment == 'Champions' else pastel_colors[i]
                                for i, segment in enumerate(segment_counts.index)],
                  marker_line_color='rgb(8, 48, 107)',
                  marker_line_width=1.5, opacity=0.6)

# Update the layout
fig.update_layout(title='Comparison of RFM Segments',
                  xaxis_title='RFM Segments',
                  yaxis_title='Number of Customers',
                  showlegend=False)

fig.show()
'''

In [ ]:
'''
# Calculate the average Recency, Frequency, and Monetary scores for each segment
segment_scores = data.groupby('RFM Customer Segments')['RecencyScore', 'FrequencyScore', 'MonetaryScore'].mean().reset_index()

# Create a grouped bar chart to compare segment scores
fig = go.Figure()

# Add bars for Recency score
fig.add_trace(go.Bar(
    x=segment_scores['RFM Customer Segments'],
    y=segment_scores['RecencyScore'],
    name='Recency Score',
    marker_color='rgb(158,202,225)'
))

# Add bars for Frequency score
fig.add_trace(go.Bar(
    x=segment_scores['RFM Customer Segments'],
    y=segment_scores['FrequencyScore'],
    name='Frequency Score',
    marker_color='rgb(94,158,217)'
))

# Add bars for Monetary score
fig.add_trace(go.Bar(
    x=segment_scores['RFM Customer Segments'],
    y=segment_scores['MonetaryScore'],
    name='Monetary Score',
    marker_color='rgb(32,102,148)'
))

# Update the layout
fig.update_layout(
    title='Comparison of RFM Segments based on Recency, Frequency, and Monetary Scores',
    xaxis_title='RFM Segments',
    yaxis_title='Score',
    barmode='group',
    showlegend=True
)

fig.show()

'''